In [1]:
import pandas as pd
import numpy as np
import json
import os
import Capstone_functions as cf
import capstone_json_functions as cjf
import sql_functions as sf
from pathlib import Path
import re


In [16]:
directory = "data/kickstarter_json/"
schema = 'kickstarter_bg'
table_name = 'boardgames'
engine = sf.get_engine()


## First try for one file
Rename files, if they start with "Kickstarter..."

In [ ]:
cjf.rename_kickstarter_files(directory)

Read the file and store it into a list called "lines"

In [ ]:
# lines is a list of strings
filename = "2022-06-09.json"
file_path = f"{directory}{filename}"
lines = cjf.list_of_lines(file_path)

build a dictionairy by looking at the "category"

In [ ]:
#list_of_categories = cjf.list_of_categories(file_path)

Build the dataframe

In [ ]:
df = cjf.build_df_from_lines_list(lines)

List of Columns to keep

In [3]:
columns_to_keep = cjf.columns_to_keep

In [4]:
columns_to_keep[0] = 'game_name'

In [5]:
columns_to_keep

['game_name',
 'goal',
 'pledged',
 'state',
 'slug',
 'country',
 'currency',
 'deadline',
 'created_at',
 'launched_at',
 'backers_count',
 'usd_pledged',
 'creator_name',
 'state_changed_at']

Overwrite the df with itself with the chosen columns

In [ ]:
df = df[columns_to_keep]

Convert Unix timestamp to pandas datetime

In [ ]:
df = cjf.unix_to_datetime(df)

# And now with all the json files in the folder data/kickstarter_json

In [ ]:
pathlist = Path(directory).glob('**/*.json')
df_all = pd.DataFrame()
counter = 0
for path in pathlist:
    file_path = str(path)
    print(file_path)
    lines = cjf.list_of_lines(file_path)
    df_temp = cjf.build_df_from_lines_list(lines)
    df_all = pd.concat([df_all, df_temp])
    counter += 1
print(f"{counter} files have been dataframed")


In [17]:
columns_to_keep = cjf.columns_to_keep
df_all = df_all[columns_to_keep]
df_all = cjf.unix_to_datetime(df_all)
df_all.rename({"name":"game_name"},axis=1,inplace=True)
df_all = df_all.astype({'usd_pledged': f"{'float64'}"}).round(2)
df_all['game_name'] =  df_all['game_name'].str.findall(r'\w|\s').str.join('').str.replace(r"\s+","_").str.lower()


/Users/ruperthufnagel/DailyLama/Daily-Lama-Capstone/capstone_json_functions.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for col in columns_to_change:
/Users/ruperthufnagel/DailyLama/Daily-Lama-Capstone/capstone_json_functions.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for col in columns_to_change:
/Users/ruperthufnagel/DailyLama/Daily-Lama-Capstone/capstone_json_functions.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [31]:
df_all

218684

In [32]:
df_without_duplicated = df_all.drop_duplicates()

In [35]:
schema = 'bgg_data'
main_table_name = 'main'
name_table = 'game_names_masked'

In [36]:
main_query = f"SELECT * FROM {schema}.{main_table_name};"
name_query = f"SELECT * FROM {schema}.{name_table};"

In [37]:
df_main_bgg = sf.get_dataframe(main_query)
df_name_bgg = sf.get_dataframe(name_query)

In [38]:
df_ks_

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206089 entries, 0 to 206088
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            206089 non-null  int64 
 1   game_name     206088 non-null  object
 2   primary_bool  206089 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.7+ MB
